In [3]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-dolly15k/resolve/main/translated-dolly-15k.json

In [4]:
import json

with open('translated-dolly-15k.json') as fopen:
    dolly = json.load(fopen)

In [5]:
import malaya

language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-ms-id')

2023-07-25 14:12:09.724500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 14:12:09.795673: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 14:12:10.284446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-25 14:12:10.284475: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [6]:
template_malaya = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [7]:
dolly[0]

{'instruction': ['In France if you were served le miel what would you eat',
  '"Di Perancis, jika anda dilayan dengan le miel, apa yang akan anda makan?"'],
 'context': '',
 'response': ['Honey', '"Madu"'],
 'category': 'open_qa'}

In [11]:
language_detection.predict([d['instruction'][1], d['response'][1]])

['standard-malay', 'local-malay']

In [27]:
from tqdm import tqdm
import random

data_dolly = []
for d in tqdm(dolly):
    try:
        langs = language_detection.predict([d['instruction'][1], d['response'][1]])
        langs = [l in ['standard-malay', 'local-malay'] for l in langs]
        if not all(langs):
            continue
        
        if random.random() > 0.5:
            q = d['instruction'][0]
        else:
            q = d['instruction'][1]

        a = {
            'prompt_input': template_malaya["prompt_no_input"],
            'input': q,
            'output': d['response'][1]
        }
        data_dolly.append(a)
    except Exception as e:
        print(e)
        pass

 34%|███████████▉                       | 5099/15015 [00:00<00:00, 25563.89it/s]

'response'


 85%|████████████████████████████▊     | 12700/15015 [00:00<00:00, 25171.34it/s]

'instruction'
'response'


100%|██████████████████████████████████| 15015/15015 [00:00<00:00, 25021.85it/s]


In [28]:
len(data_dolly)

13694

In [29]:
data_dolly[:3]

[{'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
  'input': 'In France if you were served le miel what would you eat',
  'output': '"Madu"'},
 {'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
  'input': '"Apakah makna kehidupan?"',
  'output': '"Saya tidak, pergilah baca beberapa buku."'},
 {'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
  'input': 'Frank Sinatra menukar lirik "The Lady is a Tramp" kepada "The Lady is a" apa?',
  'output': '"Juara".'}]

In [30]:
with open('prepared-dolly.jsonl', 'w') as fopen:
    for l in data_dolly:
        fopen.write(f'{json.dumps(l)}\n')